In [131]:
# Import the required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from finta import TA
from pandas.tseries.offsets import DateOffset
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout

%matplotlib inline

In [132]:
# Import indicator dataframe
# Start with 1-minute... alghough we would like to attatch them all together soon.
df = pd.read_csv("../algotrader2/resources/aapl_1min_pivot_point_indicator_df.csv")
df.head()

# df = pd.read_csv("../algotrader/resources/aapl_3min_pivot_point_indicator_df.csv")
# df.head()

# df = pd.read_csv("../algotrader/resources/aapl_1min_pivot_point_indicator_df.csv")
# df.head()



,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
0,2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,...,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0
1,2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,...,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0
2,2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,...,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0
3,2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,...,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0
4,2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,...,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0


In [133]:
# our df does not have date-time
hello = df['timestamp']
hello

0         2023-01-03 15:04:00+00:00
1         2023-01-03 15:05:00+00:00
2         2023-01-03 15:06:00+00:00
3         2023-01-03 15:07:00+00:00
4         2023-01-03 15:08:00+00:00
                    ...            
153677    2024-01-17 00:47:00+00:00
153678    2024-01-17 00:52:00+00:00
153679    2024-01-17 00:55:00+00:00
153680    2024-01-17 00:57:00+00:00
153681    2024-01-17 00:58:00+00:00
Name: timestamp, Length: 153682, dtype: object

In [134]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])

hello = df['timestamp']
hello

0        2023-01-03 15:04:00+00:00
1        2023-01-03 15:05:00+00:00
2        2023-01-03 15:06:00+00:00
3        2023-01-03 15:07:00+00:00
4        2023-01-03 15:08:00+00:00
                    ...           
153677   2024-01-17 00:47:00+00:00
153678   2024-01-17 00:52:00+00:00
153679   2024-01-17 00:55:00+00:00
153680   2024-01-17 00:57:00+00:00
153681   2024-01-17 00:58:00+00:00
Name: timestamp, Length: 153682, dtype: datetime64[ns, UTC]

In [135]:
df.set_index('timestamp', inplace=True)

df.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,127.15340,...,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0
2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,127.06139,...,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0
2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,126.94739,...,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0
2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,126.87129,...,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0
2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,126.81495,...,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0


In [136]:
# X is everything except the signal column
X = df.drop('signal', axis=1)

X.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,CCI,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,127.15340,...,-116.423590,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0
2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,127.06139,...,-101.126762,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0
2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,126.94739,...,-99.309857,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0
2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,126.87129,...,-88.964539,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0
2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,126.81495,...,-84.000283,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0


In [137]:
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()

X.head()

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,CCI,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:05:00+00:00,126.5001,126.880,126.4166,5733.0,126.820,513722.0,126.624892,-0.002516,126.87386,127.15340,...,-116.423590,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0
2023-01-03 15:06:00+00:00,126.8500,126.895,126.4400,3763.0,126.505,362900.0,126.625878,0.002766,126.80786,127.06139,...,-101.126762,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0
2023-01-03 15:07:00+00:00,126.5800,126.870,126.5300,2930.0,126.860,240388.0,126.664497,-0.002128,126.71786,126.94739,...,-99.309857,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0
2023-01-03 15:08:00+00:00,126.7100,126.870,126.5000,3189.0,126.580,289721.0,126.666956,0.001027,126.69186,126.87129,...,-88.964539,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0
2023-01-03 15:09:00+00:00,126.7066,126.760,126.5601,2861.0,126.705,255144.0,126.687038,-0.000027,126.66934,126.81495,...,-84.000283,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0


We may look to consider what it would do if we changed the amount that we shifted by. Perhaps we tried predicting 5 minutes into the future... how about an hour?

In [138]:
# y is the signal column
y = df[("signal")]

y


timestamp
2023-01-03 15:04:00+00:00    0
2023-01-03 15:05:00+00:00    0
2023-01-03 15:06:00+00:00    0
2023-01-03 15:07:00+00:00    0
2023-01-03 15:08:00+00:00    0
                            ..
2024-01-17 00:47:00+00:00    0
2024-01-17 00:52:00+00:00    0
2024-01-17 00:55:00+00:00    0
2024-01-17 00:57:00+00:00    0
2024-01-17 00:58:00+00:00    0
Name: signal, Length: 153682, dtype: int64

In [139]:
# Set start of training period
training_begin = X.index.min()

print(training_begin)

2023-01-03 15:05:00+00:00


In [140]:
# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)

print(training_end)

2023-10-03 15:05:00+00:00


In [141]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

In [142]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [143]:
display(X_train_scaled.shape)
display(X_test_scaled.shape)

(113680, 97)

(40002, 97)

In [144]:
from imblearn.over_sampling import RandomOverSampler
# Use RandomOverSampler to resample the datase using random_state=1
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

In [145]:
display(X_train_scaled.shape)
display(X_test_scaled.shape)

(113680, 97)

(40002, 97)

### Now that we have scaled our data, we can build our neural network.

In [161]:
num_predictors = len(X.columns)

# We have 3 possible outcomes, and we are trying to predict the stock/indicators to be in position -1, 0, or 1
num_classes = 1

num_predictors

97

In [162]:
nn_model = Sequential()

In [163]:
# Add dense layer(s)
nn_model.add(Dense(10, input_dim=num_predictors, activation='relu'))

/Users/montygash/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:73: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [164]:
# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))

# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))

In [165]:
# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="softmax"))

In [166]:
# Compile model
nn_model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ dense_19 (Dense)                │ (None, 10)                │        980 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_20 (Dense)                │ (None, 1)                 │         11 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 991 (3.87 KB)

 Trainable params: 991 (3.87 KB)

 Non-trainable params: 0 (0.00 B)

In [167]:
# Fit model
num_epochs = 100

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=100,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

Epoch 1/100


/Users/montygash/anaconda3/lib/python3.11/site-packages/keras/src/losses/losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 645us/step - accuracy: 0.1159 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 2/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - accuracy: 0.1160 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 3/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step - accuracy: 0.1145 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 4/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - accuracy: 0.1153 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 5/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - accuracy: 0.1136 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 6/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 366us/step - accuracy: 0.1160 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 7/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step - accuracy: 0.1145 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 8/100
910/910 ━━━━━━━━━━━━━━━━━━━

In [168]:
# Show model loss and accuracy

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1251/1251 - 0s - 283us/step - accuracy: 0.1130 - loss: 0.0000e+00
Loss: 0.0, Accuracy: 0.11304435133934021


Now we backtest with the TEST portion of the data

In [169]:
# Predict values using testing data
nn_test_predictions = nn_model.predict(X_test_scaled)
nn_train_predictions = nn_model.predict(X_train_scaled)

1251/1251 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 1s 227us/step


In [170]:
# # Evaluate the model using a classification report
# from sklearn.metrics import classification_report
# training_report = classification_report(nn_train_predictions, nn_test_predictions)
# print(training_report)

In [171]:
# # Evaluate the model's ability to predict the trading signal for the testing data using a classification report
# training_report = classification_report(y_test, testing_signal_predictions)
# print(training_report)

In [172]:
# # Save model history for further manipulation
# model_history = model.history.model_history.keys()

In [173]:
# # Now we can plot the accuracy for training and validation

# training_results = pd.DataFrame(index=range(1, num_epochs+1))
# training_results['Training'] = model_history['categorical_accuracy']
# training_results['Validation'] = model_history['val_categorical_accuracy']
# training_results.plot(title = 'Training and Validation Performance')

### Now using the 3 minute

In [174]:
# Import indicator dataframe
df = pd.read_csv("../algotrader2/resources/aapl_3min_pivot_point_indicator_df.csv")
df.head()

# df = pd.read_csv("../algotrader2/resources/aapl_1min_pivot_point_indicator_df.csv")
# df.head()

,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
0,2023-01-04 00:12:00+00:00,124.95,124.96,124.95,92,124.96,3977,124.952526,-0.00024,124.974,...,0.070644,-1.427075,1.439150,False,-5.103025e+07,0.0,5.639625,0.031817,99.378906,0
1,2023-01-04 00:15:00+00:00,124.96,124.96,124.95,83,124.95,2583,124.949560,0.00008,124.970,...,0.059256,5.886957,1.047844,False,-5.102509e+07,0.0,5.663836,0.027480,98.040465,0
2,2023-01-04 00:18:00+00:00,124.97,124.97,124.96,56,124.96,765,124.962646,0.00008,124.968,...,0.057214,12.868265,0.858268,False,-5.102356e+07,0.0,6.360573,0.024794,96.496636,0
3,2023-01-04 00:21:00+00:00,125.03,125.10,124.98,128,124.98,11318,125.033116,0.00048,124.978,...,0.073006,41.699712,1.045054,False,-5.101413e+07,0.0,6.359419,0.026735,96.194995,0
4,2023-01-04 00:24:00+00:00,125.05,125.05,125.02,38,125.04,1088,125.043639,0.00016,124.992,...,0.083011,48.064587,1.318566,False,-5.101340e+07,0.0,6.669126,0.032957,97.328048,0


In [175]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
# X is everything except the signal column
X = df.drop('signal', axis=1)
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()
display(X.head())
y = df[("signal")]
# Set start of training period
training_begin = X.index.min()
print(f"Start date: {training_begin}")
# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)
print(f"End date: {training_end}")
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,CCI,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-04 00:15:00+00:00,124.95,124.96,124.95,92.0,124.96,3977.0,124.952526,-0.00024,124.974,124.964,...,41.437189,0.070644,-1.427075,1.439150,False,-5.103025e+07,0.0,5.639625,0.031817,99.378906
2023-01-04 00:18:00+00:00,124.96,124.96,124.95,83.0,124.95,2583.0,124.949560,0.00008,124.970,124.965,...,44.639597,0.059256,5.886957,1.047844,False,-5.102509e+07,0.0,5.663836,0.027480,98.040465
2023-01-04 00:21:00+00:00,124.97,124.97,124.96,56.0,124.96,765.0,124.962646,0.00008,124.968,124.967,...,72.955382,0.057214,12.868265,0.858268,False,-5.102356e+07,0.0,6.360573,0.024794,96.496636
2023-01-04 00:24:00+00:00,125.03,125.10,124.98,128.0,124.98,11318.0,125.033116,0.00048,124.978,124.974,...,277.757872,0.073006,41.699712,1.045054,False,-5.101413e+07,0.0,6.359419,0.026735,96.194995
2023-01-04 00:27:00+00:00,125.05,125.05,125.02,38.0,125.04,1088.0,125.043639,0.00016,124.992,124.984,...,231.408886,0.083011,48.064587,1.318566,False,-5.101340e+07,0.0,6.669126,0.032957,97.328048


Start date: 2023-01-04 00:15:00+00:00
End date: 2023-10-04 00:15:00+00:00


In [176]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

display(X_train_scaled.shape)
display(X_test_scaled.shape)

from imblearn.over_sampling import RandomOverSampler
# Use RandomOverSampler to resample the datase using random_state=1
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
display(X_train_scaled.shape)
display(X_test_scaled.shape)

(47724, 97)

(16982, 97)

(47724, 97)

(16982, 97)

In [194]:
# NEURAL NETWORK
num_predictors = len(X.columns)
num_classes = 1
nn_model = Sequential()
# Add dense layer(s)
nn_model.add(Dense(10, input_dim=num_predictors, activation='relu'))
# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))
# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))
# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="softmax"))


In [195]:
# Compile model
nn_model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ dense_25 (Dense)                │ (None, 10)                │        980 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_26 (Dense)                │ (None, 1)                 │         11 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 991 (3.87 KB)

 Trainable params: 991 (3.87 KB)

 Non-trainable params: 0 (0.00 B)

In [196]:
# Fit model
num_epochs = 100

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=100,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

Epoch 1/100


/Users/montygash/anaconda3/lib/python3.11/site-packages/keras/src/losses/losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 426us/step - accuracy: 0.1149 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 2/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step - accuracy: 0.1150 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 3/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - accuracy: 0.1154 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 4/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step - accuracy: 0.1152 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 5/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step - accuracy: 0.1147 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 6/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step - accuracy: 0.1142 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 7/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 396us/step - accuracy: 0.1138 - loss: 0.0000e+00 - val_accuracy: 0.1138 - val_loss: 0.0000e+00
Epoch 8/100
910/910 ━━━━━━━━━━━━━━━━━━━

In [197]:
# Show model loss and accuracy

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1251/1251 - 0s - 237us/step - accuracy: 0.1130 - loss: 0.0000e+00
Loss: 0.0, Accuracy: 0.11304435133934021


In [198]:
# Predict values using testing data
nn_test_predictions = nn_model.predict(X_test_scaled)
nn_train_predictions = nn_model.predict(X_train_scaled)

1251/1251 ━━━━━━━━━━━━━━━━━━━━ 0s 221us/step
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 1s 229us/step


In [199]:
# # Evaluate the model using a classification report
# from sklearn.metrics import classification_report
# training_report = classification_report(nn_train_predictions, nn_test_predictions)
# print(training_report)

# # Evaluate the model's ability to predict the trading signal for the testing data using a classification report
# training_report = classification_report(y_test, testing_signal_predictions)
# print(training_report)

# # Save model history for further manipulation
# model_history = model.history.model_history.keys()

### Now 15 minute

In [200]:
# Import indicator dataframe
df = pd.read_csv("../algotrader2/resources/aapl_1min_pivot_point_indicator_df.csv")
df.head()

,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
0,2023-01-03 15:04:00+00:00,126.5001,126.880,126.4166,5733,126.820,513722,126.624892,-0.002516,126.87386,...,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0,0
1,2023-01-03 15:05:00+00:00,126.8500,126.895,126.4400,3763,126.505,362900,126.625878,0.002766,126.80786,...,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0,0
2,2023-01-03 15:06:00+00:00,126.5800,126.870,126.5300,2930,126.860,240388,126.664497,-0.002128,126.71786,...,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0,0
3,2023-01-03 15:07:00+00:00,126.7100,126.870,126.5000,3189,126.580,289721,126.666956,0.001027,126.69186,...,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0,0
4,2023-01-03 15:08:00+00:00,126.7066,126.760,126.5601,2861,126.705,255144,126.687038,-0.000027,126.66934,...,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0,0


In [201]:
df.head(100)

,timestamp,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,...,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC,signal
0,2023-01-03 15:04:00+00:00,126.5001,126.8800,126.4166,5733,126.8200,513722,126.624892,-0.002516,126.87386,...,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.000000,0
1,2023-01-03 15:05:00+00:00,126.8500,126.8950,126.4400,3763,126.5050,362900,126.625878,0.002766,126.80786,...,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.000000,0
2,2023-01-03 15:06:00+00:00,126.5800,126.8700,126.5300,2930,126.8600,240388,126.664497,-0.002128,126.71786,...,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.000000,0
3,2023-01-03 15:07:00+00:00,126.7100,126.8700,126.5000,3189,126.5800,289721,126.666956,0.001027,126.69186,...,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.000000,0
4,2023-01-03 15:08:00+00:00,126.7066,126.7600,126.5601,2861,126.7050,255144,126.687038,-0.000027,126.66934,...,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2023-01-03 16:40:00+00:00,125.3050,125.3500,125.2400,1750,125.2501,154973,125.287739,0.000599,125.16050,...,-0.051447,22.539067,0.583741,False,-1.526288e+07,0.000000,-28.352511,0.127903,100.000000,1
96,2023-01-03 16:41:00+00:00,125.2100,125.3300,125.1544,1433,125.3050,134421,125.236639,-0.000758,125.20100,...,-0.020410,9.325659,0.956244,False,-1.540833e+07,0.000000,-29.006483,0.122093,100.000000,0
97,2023-01-03 16:42:00+00:00,125.0856,125.2117,125.0600,1606,125.2100,218451,125.152599,-0.000994,125.20012,...,-0.003404,-5.500364,0.876530,False,-1.576661e+07,0.000000,-29.661408,0.120787,100.000000,0
98,2023-01-03 16:43:00+00:00,125.0596,125.1399,125.0400,1251,125.0818,117044,125.074116,-0.000208,125.17804,...,-0.034059,-8.385571,0.584045,False,-1.581863e+07,0.000000,-30.377050,0.121322,100.000000,0


In [202]:
# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
# X is everything except the signal column
X = df.drop('signal', axis=1)
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()
display(X.head())
y = df[("signal")]
# Set start of training period
training_begin = X.index.min()
print(f"Start date: {training_begin}")
# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)
print(f"End date: {training_end}")
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

,close,high,low,trade_count,open,volume,vwap,pct_returns,SMA5,SMA10,...,CCI,COPP,CMO,FISH,SQZMI,VPT,FVE,VFI,MSD,STC
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 15:05:00+00:00,126.5001,126.880,126.4166,5733.0,126.820,513722.0,126.624892,-0.002516,126.87386,127.15340,...,-116.423590,-2.878051,-65.583650,-4.458619,False,-9.952525e+06,-43.838967,-25.409400,1.034633,0.0
2023-01-03 15:06:00+00:00,126.8500,126.895,126.4400,3763.0,126.505,362900.0,126.625878,0.002766,126.80786,127.06139,...,-101.126762,-2.719298,-40.134546,-4.465828,False,-9.402193e+06,-43.321203,-24.707011,0.967187,0.0
2023-01-03 15:07:00+00:00,126.5800,126.870,126.5300,2930.0,126.860,240388.0,126.664497,-0.002128,126.71786,126.94739,...,-99.309857,-2.551580,-47.104800,-4.057573,False,-9.798127e+06,-39.135081,-24.923630,0.907692,0.0
2023-01-03 15:08:00+00:00,126.7100,126.870,126.5000,3189.0,126.580,289721.0,126.666956,0.001027,126.69186,126.87129,...,-88.964539,-2.402012,-38.479113,-3.693347,False,-9.594539e+06,-32.305140,-24.862300,0.792548,0.0
2023-01-03 15:09:00+00:00,126.7066,126.760,126.5601,2861.0,126.705,255144.0,126.687038,-0.000027,126.66934,126.81495,...,-84.000283,-2.241145,-38.583764,-3.567436,False,-9.590454e+06,-32.682043,-24.646221,0.705603,0.0


Start date: 2023-01-03 15:05:00+00:00
End date: 2023-10-03 15:05:00+00:00


In [203]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

display(X_train_scaled.shape)
display(X_test_scaled.shape)

# from imblearn.over_sampling import RandomOverSampler
# Use RandomOverSampler to resample the datase using random_state=1
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
display(X_train_scaled.shape)
display(X_test_scaled.shape)

(113680, 97)

(40002, 97)

(113680, 97)

(40002, 97)

In [204]:
# NEURAL NETWORK
num_predictors = len(X.columns)
num_classes = 1
nn_model = Sequential()
# Add dense layer(s)
nn_model.add(Dense(10, input_dim=num_predictors, activation='relu'))
# Drop-out layer(s)
# nn_model.add(Dropout(.2,input_shape=(10,)))
# Add dense layer, add Regularization
#model.add(Dense(5, activation='relu', kernel_regularized=l2(0.01), bias_regularized=l2(0.01)))
# Add output layer
# Number of outputs equals number of classes
#nn_model.add(Dense(num_classes))
nn_model.add(Dense(num_classes, activation="softmax"))


/Users/montygash/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:73: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [211]:
# Compile model
nn_model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

# Summarize model
nn_model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ dense_27 (Dense)                │ (None, 10)                │        980 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_28 (Dense)                │ (None, 1)                 │         11 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 991 (3.87 KB)

 Trainable params: 991 (3.87 KB)

 Non-trainable params: 0 (0.00 B)

In [212]:
# Fit model
num_epochs = 100

nn_model.fit(X_train_scaled, y_train,
          epochs=num_epochs,
          batch_size=100,
          validation_split=0.2,     # This 'validation_split' is telling the neural network to keep 20% of the data to validate its score on the training set... this is to help AVOID OVERFITTING. 
          shuffle=True)

Epoch 1/100


/Users/montygash/anaconda3/lib/python3.11/site-packages/keras/src/losses/losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


910/910 ━━━━━━━━━━━━━━━━━━━━ 1s 471us/step - accuracy: 0.7554 - loss: 0.0000e+00 - val_accuracy: 0.7478 - val_loss: 0.0000e+00
Epoch 2/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step - accuracy: 0.7631 - loss: 0.0000e+00 - val_accuracy: 0.7551 - val_loss: 0.0000e+00
Epoch 3/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - accuracy: 0.7678 - loss: 0.0000e+00 - val_accuracy: 0.7640 - val_loss: 0.0000e+00
Epoch 4/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 386us/step - accuracy: 0.7703 - loss: 0.0000e+00 - val_accuracy: 0.7685 - val_loss: 0.0000e+00
Epoch 5/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 357us/step - accuracy: 0.7729 - loss: 0.0000e+00 - val_accuracy: 0.7716 - val_loss: 0.0000e+00
Epoch 6/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 357us/step - accuracy: 0.7701 - loss: 0.0000e+00 - val_accuracy: 0.7734 - val_loss: 0.0000e+00
Epoch 7/100
910/910 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step - accuracy: 0.7701 - loss: 0.0000e+00 - val_accuracy: 0.7740 - val_loss: 0.0000e+00
Epoch 8/100
910/910 ━━━━━━━━━━━━━━━━━━━

In [213]:
# Show model loss and accuracy

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1251/1251 - 0s - 241us/step - accuracy: 0.7745 - loss: 0.0000e+00
Loss: 0.0, Accuracy: 0.7745112776756287


In [214]:
# Predict values using testing data
nn_test_predictions = nn_model.predict(X_test_scaled)
nn_train_predictions = nn_model.predict(X_train_scaled)

1251/1251 ━━━━━━━━━━━━━━━━━━━━ 0s 220us/step
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 1s 201us/step


In [220]:
nn_test_predictions[:500]

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [215]:
# # Evaluate the model using a classification report
# from sklearn.metrics import classification_report
# training_report = classification_report(nn_train_predictions, nn_test_predictions)
# print(training_report)

# # Evaluate the model's ability to predict the trading signal for the testing data using a classification report
# training_report = classification_report(y_test, testing_signal_predictions)
# print(training_report)

# # Save model history for further manipulation
# model_history = model.history.model_history.keys()

In [216]:
# # Deployment (Hypothetical)
# while True:
#     current_data = your_trading_api.get_real_time_data()
#     current_data_processed = preprocess_data(current_data)
#     prediction = model.predict(current_data_processed)
#     if prediction > some_threshold:
#         your_trading_api.execute_trade()

# # Placeholder Functions
# def combine_data(historical, news):
#     # Combine and return data
#     pass

# def split_data(data):
#     # Split and return data
#     pass

# def backtest_strategy(model, data):
#     # Implement backtesting logic
#     pass

# def preprocess_data(data):
#     # Data preprocessing steps
#     pass